In [ ]:
!pip install pyngrok flask
!pip install pydub librosa

^C


In [ ]:
from pyngrok import ngrok
from flask import Flask, request, send_file
import librosa
from pydub import AudioSegment
import io
import os
from google.colab import userdata


app = Flask(__name__)

# Get auth token from ngrok dashboard and authenticate
ngrok.set_auth_token(userdata.get('NGROK_SECRET'))

# connect
public_url = ngrok.connect(5000)
print(f' * Public URL: {public_url}')

@app.route('/', methods=['GET'])
def home():
    return "Audio processing API is running!"

@app.route('/process-audio', methods=['POST'])
def process_audio():
    if 'file' not in request.files:
        return {'error': 'No file provided'}, 400

    file = request.files['file']
    audio_data = file.read()
    audio_io = io.BytesIO(audio_data)

    # Save temporarily
    # temp_path = 'temp_audio' # needs to handle file extension
    # file.save(temp_path)

    try:
        # Process audio
        y, sr = librosa.load(audio_io)

        # Detect pitch
        f0, voiced_flag = librosa.pyin(y, fmin=150, fmax=2000, sr=sr)
        voiced_f0 = f0[voiced_flag.astype(bool)]
        if len(voiced_f0) > 0:
            target_pitch = librosa.hz_to_note(voiced_f0.mean())

        # Convert to AudioSegment
        audio = AudioSegment.from_file(audio_io)

        # Normalize
        normalized = audio.normalize()

        # Trim silence
        trimmed = normalized.strip_silence()

        # Save processed audio
        output_path = 'processed_audio.wav'
        trimmed.export(output_path, format='wav')

        # Return file
        response = send_file(output_path, mimetype='audio/wav')

        # Clean up temporary files
        #os.remove(temp_path)
        os.remove(output_path)

        return response

    except Exception as e:
        # Clean up temporary files in case of error
        if os.path.exists('processed_audio.wav'):
            os.remove('processed_audio.wav')

        print(f"Error: {str(e)}")
        return {'error': str(e)}, 500

app.run()

 * Public URL: NgrokTunnel: "https://c0e5-34-86-37-114.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Dec/2024 19:55:40] "POST /process-audio HTTP/1.1" 500 -


Error: too many values to unpack (expected 2)
